In [ ]:
import pandas as pd

def main():

    hierarquizacao = pd.read_excel("Trabalho 1 - Deadline 24_05 (2).xlsx", sheet_name="Página7")
    infos = pd.read_excel("Trabalho 1 - Deadline 24_05 (2).xlsx", sheet_name="Página8")

    #display(hierarquizacao)

    #display(infos)

    roteiros = {}
    roteiro = []

    CAP = 2800
    TEMPO = 8*60
    KM_H = 60
    
    cont = 0
    
    resposta = criar_roteiro(hierarquizacao, infos, roteiros, roteiro, CAP, TEMPO, KM_H, cont)
    print(resposta)
    
    
def ajustar_df(num, df):
    for i in range(num + 1):
        df = df.drop(i)
        
    return df
    
    
def criar_roteiro(hierarquizacao, infos, roteiros, roteiro, CAP, TEMPO, KM_H, cont):
    
    cont += 1

    for linha in hierarquizacao.index:

        # isolar números da nomenclatura
        nome = hierarquizacao.loc[linha, "nomenclatura"]
        nome = nome.replace("S_", "")
        nums = nome.split("_")

        # se for primeira linha
        if linha == hierarquizacao.head(1).index.item():

            roteiro.append(0)

            # pegar carga total do percurso
            carga = 0
            for n in nums:
                carga += infos.loc[infos["Número"] == int(n),"Kg de produto/semana"].item()

            # checar restrições
            if carga <= CAP:

                # pegar distâncias entre os pontos
                dist_0_num = infos.loc[infos["Número"] == int(nums[0]), 0].item()
                dist_num_outro = infos.loc[infos["Número"] == int(nums[0]), int(nums[1])].item()
                dist_outro_0 = infos.loc[infos["Número"] == int(nums[1]), 0].item()

                dist = dist_0_num + dist_num_outro + dist_outro_0

                temp_desloc = dist / KM_H
                temp_desc = infos.loc[infos["Número"] == int(nums[0]), "Tempo de Descarga (min.)"].item() + infos.loc[infos["Número"] == int(nums[1]), "Tempo de Descarga (min.)"].item()
                tempo = temp_desloc + temp_desc

                if tempo <= TEMPO:
                    roteiro.append(nums[0])
                    roteiro.append(nums[1])
                    roteiro.append(0)
                    roteiros[cont] = roteiro.copy()
                    roteiro.clear()

        # se for diferente da primeira linha
        else:

            # percorrer dicionário previamente para ver se valores já existem em algum roteiro
            tem_um = tem_outro = tem_dois = False

            for rot in roteiros:

                # se ambos já estiverem na lista
                if nums[0] in roteiros[rot] and nums[1] in roteiros[rot]:
                    tem_dois = True

                elif nums[0] in roteiros[rot]:
                    tem_um = True

                elif nums[1] in roteiros[rot]:
                    tem_outro = True

            for rot2 in roteiros:
                rot_atual = roteiros[rot2]

                # se já tiver os dois, pular
                if tem_dois:
                    break

                # se apenas um dos dois estiver na lista
                elif nums[0] in rot_atual and tem_outro is False:

                    # verificar se há existência de 0 à esquerda ou à direita
                    if rot_atual[rot_atual.index(nums[0]) - 1] == 0:
                        roteiros[cont].insert(1, nums[1])

                        # verificar restrições
                        pontos_meio = rot_atual[1:-1]

                        carga = 0
                        for m in pontos_meio:
                            carga += infos.loc[infos["Número"] == int(m),"Kg de produto/semana"].item()

                        if carga <= CAP:

                            # distância do depósito para o ponto inicial e para o ponto final
                            dist_0_num = infos.loc[infos["Número"] == int(rot_atual[1]), 0].item()
                            dist_outro_0 = infos.loc[infos["Número"] == int(rot_atual[-2]), 0].item()

                            dist_entre = 0
                            for i in range(len(pontos_meio) - 1):
                                dist_entre += infos.loc[infos["Número"] == int(pontos_meio[i]), int(pontos_meio[i + 1])].item()

                            dist = dist_0_num + dist_outro_0 + dist_entre

                            temp_desloc = dist / KM_H

                            temp_desc = 0
                            for desc in pontos_meio:
                                temp_desc += infos.loc[infos["Número"] == int(desc), "Tempo de Descarga (min.)"].item()

                            tempo = temp_desloc + temp_desc

                            if tempo <= TEMPO:
                                break

                            else:
                                roteiros[cont].pop(1)
                                break
                        else:
                            roteiros[cont].pop(1)
                            break

    

                    elif rot_atual[rot_atual.index(nums[0]) + 1] == 0:
                        roteiros[cont].insert(-2, nums[1])

                        # verificar restrições
                        pontos_meio = rot_atual[1:-1]

                        carga = 0
                        for m in pontos_meio:
                            carga += infos.loc[infos["Número"] == int(m),"Kg de produto/semana"].item()

                        if carga <= CAP:

                            # distância do depósito para o ponto inicial e para o ponto final
                            dist_0_num = infos.loc[infos["Número"] == int(rot_atual[1]), 0].item()
                            dist_outro_0 = infos.loc[infos["Número"] == int(rot_atual[-2]), 0].item()

                            dist_entre = 0
                            for i in range(len(pontos_meio) - 1):
                                dist_entre += infos.loc[infos["Número"] == int(pontos_meio[i]), int(pontos_meio[i + 1])].item()

                            dist = dist_0_num + dist_outro_0 + dist_entre

                            temp_desloc = dist / KM_H

                            temp_desc = 0
                            for desc in pontos_meio:
                                temp_desc += infos.loc[infos["Número"] == int(desc), "Tempo de Descarga (min.)"].item()

                            tempo = temp_desloc + temp_desc

                            if tempo <= TEMPO:
                                break

                            else:
                                roteiros[cont].pop(-2)
                                break
                        else:
                            roteiros[cont].pop(-2)
                            break


                # se apenas um dos dois estiver na lista
                elif nums[1] in rot_atual and tem_um is False:

                    # verificar se há existência de 0 à esquerda ou à direita
                    if rot_atual[rot_atual.index(nums[1]) - 1] == 0:
                        roteiros[cont].insert(1, nums[0])


                        # verificar restrições
                        pontos_meio = rot_atual[1:-1]

                        carga = 0
                        for m in pontos_meio:
                            carga += infos.loc[infos["Número"] == int(m),"Kg de produto/semana"].item()

                        if carga <= CAP:

                            # distância do depósito para o ponto inicial e para o ponto final
                            dist_0_num = infos.loc[infos["Número"] == int(rot_atual[1]), 0].item()
                            dist_outro_0 = infos.loc[infos["Número"] == int(rot_atual[-2]), 0].item()

                            dist_entre = 0
                            for i in range(len(pontos_meio) - 1):
                                dist_entre += infos.loc[infos["Número"] == int(pontos_meio[i]), int(pontos_meio[i + 1])].item()

                            dist = dist_0_num + dist_outro_0 + dist_entre

                            temp_desloc = dist / KM_H

                            temp_desc = 0
                            for desc in pontos_meio:
                                temp_desc += infos.loc[infos["Número"] == int(desc), "Tempo de Descarga (min.)"].item()

                            tempo = temp_desloc + temp_desc

                            if tempo <= TEMPO:
                                break

                            else:
                                roteiros[cont].pop(1)
                                break
                        else:
                            roteiros[cont].pop(1)
                            break


                    elif rot_atual[rot_atual.index(nums[1]) + 1] == 0:
                        roteiros[cont].insert(-2, nums[0])

                        # verificar restrições
                        pontos_meio = rot_atual[1:-1]

                        carga = 0
                        for m in pontos_meio:
                            carga += infos.loc[infos["Número"] == int(m),"Kg de produto/semana"].item()

                        if carga <= CAP:

                            # distância do depósito para o ponto inicial e para o ponto final
                            dist_0_num = infos.loc[infos["Número"] == int(rot_atual[1]), 0].item()
                            dist_outro_0 = infos.loc[infos["Número"] == int(rot_atual[-2]), 0].item()

                            dist_entre = 0
                            for i in range(len(pontos_meio) - 1):
                                dist_entre += infos.loc[infos["Número"] == int(pontos_meio[i]), int(pontos_meio[i + 1])].item()

                            dist = dist_0_num + dist_outro_0 + dist_entre

                            temp_desloc = dist / KM_H

                            temp_desc = 0
                            for desc in pontos_meio:
                                temp_desc += infos.loc[infos["Número"] == int(desc), "Tempo de Descarga (min.)"].item()

                            tempo = temp_desloc + temp_desc

                            if tempo <= TEMPO:
                                break

                            else:
                                roteiros[cont].pop(-2)
                                break
                        else:
                            roteiros[cont].pop(-2)
                            break

                # se nenhum dos dois estiver no roteiro
                else:
                    
                    # diminuir linhas
                    ajustar_df(linha, hierarquizacao)
                    
                    # fazer uma recursiva
                    criar_roteiro(hierarquizacao, infos, roteiros, roteiro, CAP, TEMPO, KM_H, cont)

                    
    return roteiros
               
    
if __name__ == "__main__":
    main()
    


In [61]:
teste = hierarquizacao.copy()

display(teste)

for i in range(12):
    teste = teste.drop(i)
    
display(teste)

print(teste.head(1).index.item())

,nomenclatura,ganho
0,S_10_11,40.2
1,S_7_10,40.0
2,S_9_10,39.5
3,S_7_11,39.2
4,S_6_10,39.2
...,...,...
100,S_3_15,25.4
101,S_11_15,24.7
102,S_7_15,24.6
103,S_13_15,23.0


,nomenclatura,ganho
12,S_6_13,37.8
13,S_10_13,37.8
14,S_7_9,37.7
15,S_10_12,37.5
16,S_12_13,37.4
...,...,...
100,S_3_15,25.4
101,S_11_15,24.7
102,S_7_15,24.6
103,S_13_15,23.0


12
